<div align="center" style="font-family: 'Segoe UI', sans-serif;">

# 🧬 **Molecule Visualizer with Topological Indices** 🧬
</div>

This Jupyter notebook is an interactive tool for visualizing molecular structures from `.mol` and `.sdf` files and calculating graph-based topological indices such as:

- 📊 **Edge Density**: The ratio of actual edges to the maximum possible in the graph.
- 📏 **Wiener Index**: The sum of shortest path lengths between all non-hydrogen pairs of vertices.
- ⚖️ **Petitjean Index**: A normalized measure of asymmetry based on diameter and radius.
<div align="right" style="font-size:16px; color:black; font-family:Segoe UI, sans-serif;">
Created by: Haya Mazyad & Lynn Oueidat
</div>

---


In [ ]:
%pip install plotly networkx ipywidgets ipyfilechooser


### ▶️ How to Use

1. **Run all cells** starting from the top.
2. Use the **file chooser** to select a `.mol`, `.sdf`, or folder containing these files.
3. Choose your desired **layout (2D or 3D)** and **index type**.
4. Click **Compute Indices** to visualize the molecule(s).
5. Use **Previous/Next** to navigate between multiple molecules.
6. Click **Export Indices💾** to save the results for the current molecule.
7. Use the small 📷 button in the top-right corner of the graph to **download the 3D plot as a PNG**.

Before we start, make sure all required packages are installed (see the first cell).


In [ ]:
import os
import glob
import networkx as nx
import plotly.graph_objects as go
from ipywidgets import VBox, HBox, Layout, Button, HTML, Dropdown, Label, Text, Output
from ipyfilechooser import FileChooser
from IPython.display import display, clear_output
import numpy as np
from datetime import datetime
from collections import deque

### ✅ Test Cases & Validation

To validate the script, we tested it using the following files:

| File Name                      | Format | Description                            |
|--------------------------------|--------|----------------------------------------|
| `Octamethylenediamine.mol`     | `.mol` | Basic molecule with single bonds       |
| `Benzene.mol`                  | `.mol` | Cyclic aromatic molecule               |
| `Imatinib.sdf`                 | `.sdf` | Molecule found in the slides           |
| `Drugs.sdf`                    | `.sdf` | Multiple chembl molecules in one file  |
| `MolFiles`                     | `.mol` | Folder containing several mol files    |

The output matched expected indices when calculated manually, confirming correctness. We also tested it on the molecule Imatinib to validate the output and obtained the same results as those presented in the slides.


### ✨ Additional Features Implemented

- **Export to `.txt`**: Users can save the computed indices for any molecule.
- **Interactive 2D/3D View Toggle**: Users can switch between true 3D coordinates or a force-directed layout.
- **Styled UI Panels**: Includes responsive layout, styled index panel with colors and emojis.
- **Bond Multiplicity Visualization**: Double/triple bonds rendered with proper spacing using offset logic.
- **Tooltips**: Atom hover text includes index, element type, and 3D coordinates.


### **Global Variables and UI Elements**
This section initializes global variables and UI components for the molecule visualization tool. It sets up:
- **`graphs_data`** to store molecule graph data.
- **`current_index`** to track the current molecule.
- **`current_fig`** to hold the current graph figure.

It also defines UI elements for displaying results, selecting the layout (2D or 3D), and choosing the topological index to visualize.

In [ ]:
# Global variables and UI elements
graphs_data = []  # Stores molecule graph data.
current_index = 0  
current_fig = None  # Holds the current graph figure.

# Make output areas expand to full width
output_area = Output()  # General output area for results.

# Styling indicies output area (border, padding, hidden by default, etc.)
indices_output = Output(layout=Layout(
    border='2px solid #3973ac', padding='15px', display='none', margin='120px 0 0 10px', 
    width='320px', background_color='white', font_family='monospace', box_shadow='0 4px 8px rgba(0, 0, 0, 0.1)'
))


# UI components
mol_count_label = Label()  # Label for molecule count or info.
view_dropdown = Dropdown(
    options=['2D Layout', '3D XYZ'],  # Layout options 2D or 3D
    value='2D Layout',  # Default layout
    description='View:'
)
index_dropdown = Dropdown(
    options=['Edge Density', 'Wiener Index', 'PetitJean Index', 'All'],  # Index options
    value='All',  # Default index
    description='Index:'
)

### **Molecule Parsing and Graph Extraction**
This code parses `.mol` and `.sdf` files to extract molecule data, including atom coordinates, bond types, and element symbols. 
The `parse_molecule_from_block` function extracts graph data, names, and 3D coordinates, which are then used to build a molecular graph using the `networkx` library. 
The `parse_mol_file` and `process_sdf_file` functions are used to process single files or entire directories of molecules, storing the graph data for further analysis.

In [ ]:
# Parses a MOL/SDF block to extract molecule graph, name, and 3D coordinates
def parse_molecule_from_block(block):
    lines = block.strip().splitlines()  # Split block into lines
    graph = nx.Graph()  # Initialize an empty graph 
    coords = {}  # Store atom coordinates
    name = lines[0].strip()  # Get the molecule name from the first line
    
    # Parse atom and bond data from the block
    for i, line in enumerate(lines):
        if 'V2000' in line:  # Start parsing atom/bond data in & after  V2000 line
            atom_count = int(line[:3])
            bond_count = int(line[3:6])
            atom_lines = lines[i+1:i+1+atom_count]
            bond_lines = lines[i+1+atom_count:i+1+atom_count+bond_count]
            break
    
    # Add atoms to the graph with their coordinates and elements
    for idx, atom_line in enumerate(atom_lines):
        x, y, z, element = atom_line[:10], atom_line[10:20], atom_line[20:30], atom_line[31:34].strip() # Get the coordinates
        atom_id = idx + 1
        graph.add_node(atom_id, element=element)  # Add atom as node
        coords[atom_id] = (float(x), float(y), float(z))  # Store coordinates
    
    # Add bonds to the graph
    for bond in bond_lines:
        start, end = int(bond[:3]), int(bond[3:6]) # Get the start and end of a bond
        bond_type = int(bond[6:9]) # Get the ttpe of the bond
        graph.add_edge(start, end, bond_type=bond_type)  # Add bond as edge
    
    # Extract molecule name if present in the block
    # In this part, if the file is a Chembl file, it will take the Chembl name, but if it's not, it will take the first line as the name or id
    for i, line in enumerate(lines):
        if '> <chembl_pref_name>' in line and i + 1 < len(lines):
            name = lines[i + 1].strip()  # Update name if found
    
    return graph, name, coords  # Return graph, name, and coordinates

# Reads and parses a single .mol file, returning graph, name, and coordinates
def parse_mol_file(file_path):
    with open(file_path, 'r') as f:
        content = f.read()  # Read file content
    return parse_molecule_from_block(content)  # Parse the molecule data

# Splits and processes all molecule blocks in an SDF file
def process_sdf_file(path):
    with open(path, 'r') as f:
        blocks = f.read().split('$$$$\n')  # Split the file into blocks
    for block in blocks:
        if block.strip():
            graphs_data.append(parse_molecule_from_block(block))  # Parse and append each molecule in the sdf file

# Determines the file format and processes the molecules into graph data
def compute_indices(input_path):
    graphs_data.clear()  # Clear existing graph data everytime we choose another file
    if os.path.isdir(input_path):  # If input is a directory
        for f in glob.glob(os.path.join(input_path, '*.mol')):  # Process all .mol files in the directory/ folder
            graphs_data.append(parse_mol_file(f))
        for f in glob.glob(os.path.join(input_path, '*.sdf')):  # Process all .sdf files in the directory
            process_sdf_file(f)
    elif input_path.endswith('.mol'):  # Process a single given .mol file
        graphs_data.append(parse_mol_file(input_path))
    elif input_path.endswith('.sdf'):  # Process a single given .sdf file
        process_sdf_file(input_path)

### **Topological Index Calculations**
This section of the code defines functions to calculate various topological indices for molecular graphs. 
The `EdgeDensity` function computes the edge density, the `WienerIndex` function calculates the Wiener index, 
and the `PetitJeanIndex` measures graph asymmetry. Additional functions like `shortest_path` and `calculate_diameter_and_radius` are used to support these calculations 
by computing the shortest paths and the graph's diameter and radius.  
To ensure accurate and chemically meaningful results, hydrogen atoms are excluded from all index calculations. However, hydrogens are still included in the graph visualization output to preserve the full molecular structure.

In [ ]:
# Calculates the edge density of the graph (measure of graph sparsity)
def EdgeDensity(graph):
    # Create subgraph with only heavy atoms (non-H)
    heavy_atoms = [n for n in graph.nodes() if graph.nodes[n].get('element') != 'H']
    subgraph = graph.subgraph(heavy_atoms)

    num_edges = subgraph.number_of_edges()
    num_vertices = subgraph.number_of_nodes()

    return (2 * num_edges) / (num_vertices * (num_vertices - 1)) if num_vertices > 1 else 0

# Computes shortest path lengths from a start node using Breadth-First Search
def shortest_path(graph, start_node):
    distances = {start_node: 0}  # Initialize distances dictionary with start node
    queue = deque([start_node])  # Queue for BFS
    while queue:  # BFS loop
        current = queue.popleft()  # Dequeue node
        for neighbor in graph.neighbors(current):  # Traverse neighbors
            if neighbor not in distances:  # Check if the neighbor is unvisited
                distances[neighbor] = distances[current] + 1  # Update distance
                queue.append(neighbor)  # Enqueue the neighbor
    return distances  # Return distances from start node

# Calculates the Wiener Index (sum of shortest paths between all non-hydrogen node pairs)
def WienerIndex(graph):
    total = 0
    # Filter non-hydrogen atoms
    heavy_atoms = [n for n in graph.nodes() if graph.nodes[n]['element'] != 'H']
    
    # Sum shortest paths between each heavy atom and all others
    for node in heavy_atoms:
        sp_lengths = shortest_path(graph, node)
        total += sum(dist for target, dist in sp_lengths.items() if target in heavy_atoms)

    return total // 2  # Divide by 2 to avoid double-counting

# Computes the graph's diameter and radius based on eccentricities of nodes
def calculate_diameter_and_radius(graph):
    heavy_atoms = [n for n in graph.nodes() if graph.nodes[n].get('element') != 'H']
    subgraph = graph.subgraph(heavy_atoms)
    
    eccentricities = []
    for node in subgraph.nodes():
        distances = shortest_path(subgraph, node)  # your BFS
        ecc = max(distances.values())
        eccentricities.append(ecc)

    return max(eccentricities), min(eccentricities)

# Computes the Petitjean Index  using diameter and radius
def PetitJeanIndex(graph):
    diameter, radius = calculate_diameter_and_radius(graph)  # Get diameter and radius
    return (diameter - radius) / radius if radius != 0 else 0  # Calculate and return Petitjean Index


### **Atom Color and Perpendicular Offsets**
Here, the code provides utility functions for visualizing molecules. 
The `atom_color` function assigns colors to atoms based on their element type (e.g., red for oxygen, blue for nitrogen). 
The `unit_vector` function normalizes a vector to unit length, and `perpendicular_offset` calculates a perpendicular vector used to offset bond lines for rendering multi-bond connections in the 3D graph.

In [ ]:
# Returns a color based on the atomic element symbol (for visual representation)
def atom_color(element):
    colors = {'C': 'gray', 'O': 'red', 'N': 'blue', 'H': 'beige', 'S': 'yellow', 'Cl': 'green'}  # Atomic element to color mapping
    return colors.get(element, 'purple')  # Return the color, default to 'purple' if the element is not in the mapping

# Normalizes a vector to unit length (to ensure consistent vector magnitude)
def unit_vector(v):
    norm = np.linalg.norm(v)  # Calculate the magnitude of the vector
    return v / norm if norm > 0 else v  # Return the unit vector, or the original if norm is 0

# Computes a perpendicular offset for multi-bond rendering (used for drawing bond angles)
def perpendicular_offset(v):
    perp = np.cross(v, [0, 0, 1])  # Compute the perpendicular vector to 'v' in the XY plane
    if np.linalg.norm(perp) == 0:  # If the perpendicular is zero, compute a different perpendicular vector
        perp = np.cross(v, [0, 1, 0])
    return unit_vector(perp) * 0.07  # Normalize the perpendicular and scale by a factor (0.07) for bond offset

### **3D Graph Visualization and Index Display**
This code generates a 3D visualization of a molecular graph, displaying atoms as colored spheres and bonds as lines between nodes. 
It uses the Plotly library to render the graph in 3D space. The graph layout can be either 2D or 3D based on user selection. 
It also displays the molecular name, atom details (with hover text), and provides the option to view and calculate topological indices (like Edge Density, Wiener Index, and PetitJean Index). 
The figure is then displayed in a UI component.

In [ ]:
# Generates and displays the 3D graph visualization with topological indices
def plot_graph(index):
    global current_fig  
    g, name, coords = graphs_data[index]  # Get graph, name, and coordinates for the selected molecule
    layout_mode = view_dropdown.value  # Get the selected layout (2D or 3D)
    
    # Set position of nodes based on the selected layout mode
    pos = coords if layout_mode == '3D XYZ' else {k: (v[0], v[1], 0) for k, v in nx.kamada_kawai_layout(g).items()}

    # Initialize lists for node positions, colors, labels, and hover texts
    node_x, node_y, node_z, hover_texts, colors, labels = [], [], [], [], [], []
    for n in g.nodes():  # Iterate over each node in the graph
        x, y, z = pos[n]  # Get node coordinates
        atom = g.nodes[n]['element']  # Get atom element
        node_x.append(x)
        node_y.append(y)
        node_z.append(z) # Add the coordinates
        hover_texts.append(f"Atom {n} ({atom})<br>X={x:.2f}, Y={y:.2f}, Z={z:.2f}")  # Hover text with coordinates
        labels.append(atom)  # Atom labels for nodes
        colors.append(atom_color(atom))  # Assign color based on atom type

    fig = go.Figure()  # Initialize figure for 3D plotting

    # Add edges (bonds) to the figure with proper offsets for multi-bond rendering
    for u, v, data in g.edges(data=True):
        x0, y0, z0 = pos[u] # Coordinates of the start node of the bond
        x1, y1, z1 = pos[v] # Coordinates of the end node of the bond
        bond_type = data.get('bond_type', 1)  # Get bond type (default to 1)
        direction = np.array([x1 - x0, y1 - y0, z1 - z0])  # Calculate bond direction
        offset = perpendicular_offset(direction)  # Compute offset for multi-bond rendering
        for i in range(bond_type):  # To add multiple bonds if needed
            shift = (i - (bond_type - 1) / 2)  # Distribute bonds
            dx, dy, dz = offset * shift  # Apply the offset
            fig.add_trace(go.Scatter3d(  # Add bond line to figure
                x=[x0+dx, x1+dx], y=[y0+dy, y1+dy], z=[z0+dz, z1+dz],
                mode='lines', line=dict(color='gray', width=2), hoverinfo='none'))

    # Add nodes to the figure as scatter points with labels and hover info
    fig.add_trace(go.Scatter3d(
        x=node_x, y=node_y, z=node_z,
        mode='markers+text', marker=dict(size=6, color=colors),
        text=labels, hovertext=hover_texts, hoverinfo='text'))  # Display atom info on hover

    # The layout settings for the figure/ graph
    fig.update_layout(
        title=name, title_font=dict(size=24, family='Arial', color='#1a237e'),
        showlegend=False, margin=dict(l=0, r=0, t=70, b=0),
        scene=dict(xaxis=dict(visible=False), yaxis=dict(visible=False), zaxis=dict(visible=False), bgcolor='white')
    )

    current_fig = fig  # Store the current figure
    with output_area:  # Display the figure in the output area
        clear_output()  # Clear previous output figure
        display(fig)  # and display the new one

    with indices_output:  # Display topological indices
        clear_output()  # Clear previous indices
        indices_output.layout.display = 'block'  # Show indices output area
        index_choice = index_dropdown.value  # Get selected index option
        html = '<div style="font-size: 15px; color: #2c3e50;">'
        html += '<div style="font-weight: bold; font-size: 16px; margin-bottom: 8px; color: #1a237e;">🧠 Topological Indices:</div>'

        # Display selected topological indices (Edge Density, Wiener Index, PetitJean Index)
        if index_choice == 'All' or index_choice == 'Edge Density':
            html += f'<div title="The ratio of actual edges to the maximum possible in the graph.">📊 Edge Density: <span style="font-weight: bold;">{EdgeDensity(g):.2f}</span></div>'

        if index_choice == 'All' or index_choice == 'Wiener Index':
            html += f'<div title="The sum of shortest path lengths between all non-hydrogen pairs of vertices.">📏 Wiener Index: <span style="font-weight: bold;">{WienerIndex(g)}</span></div>'

        if index_choice == 'All' or index_choice == 'PetitJean Index':
            html += f'<div title="A normalized measure of asymmetry based on diameter and radius.">⚖️ Petitjean Index: <span style="font-weight: bold;">{PetitJeanIndex(g):.2f}</span></div>'

        html += '</div>'
        display(HTML(html))  # Display the indices in HTML format, hover over each index to get its descrition

    # Update molecule count label (e.g., "Molecule 1 of 10")
    mol_count_label.value = f"Molecule {index+1} of {len(graphs_data)}"


### **Topological Index Export**
This section handles the export of calculated topological indices (e.g., Edge Density, Wiener Index, PetitJean Index) to a text file. 
It checks whether a molecule is loaded and creates a `.txt` file with the index values. 

In [ ]:
# Exports the topological indices of the current molecule to a .txt file
def on_export_indices(b):
    if not graphs_data:  # Check if no molecules are loaded, display a no molecule loaded text
        mol_count_label.value = "⚠ No molecule loaded."
        return
    g, name, _ = graphs_data[current_index]  # Get the current molecule's graph and name
    filename = f"{name.replace(' ', '_')}_indices.txt"  # Generate a filename based on the molecule's name
    edge = EdgeDensity(g)  # Calculate edge density
    wiener = WienerIndex(g)  # Calculate Wiener index
    petit = PetitJeanIndex(g)  # Calculate PetitJean index
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")  # Get current timestamp of when the file text is exported
    content = f"""Molecule: {name}\nExported: {timestamp}\n\n📊 Edge Density: {edge:.2f}\n📏 Wiener Index: {wiener}\n⚖️ Petitjean Index: {petit:.2f}\n"""  # Create content string to add in the text file

    try:
        # Try to write the indices data to a file
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(content) # writing the content string
        mol_count_label.value = f"✅ Indices saved to '{filename}'"  # Success message
    except Exception as e:
        mol_count_label.value = f"❌ Failed to save file: {e}"  # Error message if file saving fails


### **User Interface Setup and Interaction**
This chunk sets up the user interface, including buttons for navigating through molecules, computing indices, and exporting results. 
The interface includes a file chooser for selecting molecule files, text input for manual path entry, and dropdown menus for selecting the layout and index type. 
The controls are arranged in a vertical box layout, and the main UI is displayed using a horizontal box layout. 
The UI allows users to interact with the application, visualize molecule graphs, and compute and export topological indices.

In [ ]:
# Displays the previous molecule in the dataset
def on_prev(b):
    """Navigate to the previous molecule."""
    global current_index
    if current_index > 0:  # Check if there's a previous molecule
        current_index -= 1 # go to the previous molecule and set it as the current
        plot_graph(current_index)  # Plot the previous molecule

# Displays the next molecule in the dataset
def on_next(b):
    """Navigate to the next molecule."""
    global current_index
    if current_index < len(graphs_data) - 1:  # Check if there's a next molecule
        current_index += 1 # go to the next molecule and set it as the current
        plot_graph(current_index)  # Plot the next molecule

# File chooser for selecting a molecule file or directory
file_chooser = FileChooser()
file_chooser.title = 'Choose .mol/.sdf file or directory:'
file_chooser.use_dir_icons = True

# Text box for manually pasting file paths
manual_path = Text(placeholder='Or paste path here...', layout=Layout(width='50%'))

# Button to compute the indices
compute_btn = Button(description='Compute Indices', button_style='success', layout=Layout(margin='15px 0 15px 0'))

# Buttons to navigate between molecules (previous and next buttons) and export indices
prev_btn = Button(description='⬅️ Previous')
next_btn = Button(description='Next ➡️')
export_indices_btn = Button(description='Export Indices💾')

# Assign actions to buttons
prev_btn.on_click(on_prev)
next_btn.on_click(on_next)
export_indices_btn.on_click(on_export_indices)

# Handles input selection, parses molecules, and starts visualization
def on_compute(b):
    """Load and visualize the selected molecule(s)."""
    global current_index
    path = ''  # Initialize the file path
    if file_chooser.selected:  # Check if a file is selected from the file chooser
        path = file_chooser.selected
    elif manual_path.value:  # Check if a manual path is provided
        path = manual_path.value
    if not path:  # If no path is selected, show an error
        mol_count_label.value = 'No file or path selected.'
        return
    compute_indices(path)  # Compute the indices for the selected file(s)
    current_index = 0  # Set the initial index to the first molecule
    if graphs_data:  # Check if molecules are loaded
        plot_graph(current_index)  # Plot the first molecule
    else:
        mol_count_label.value = 'No valid molecules found.'  # Show an error if no valid molecules are found

# Assign the compute function to the button's on_click event
compute_btn.on_click(on_compute)

# Layout and structure for the user interface
controls = VBox([
    HTML('<h2 style="font-size:28px; color:#2c3e50; text-shadow: 1px 1px #ccc;">🧬 Molecule Visualizer with Topological Indices 🧬</h2>'), # title of the output page
    file_chooser,  # File chooser for selecting .mol/.sdf files
    manual_path,  # Text box for manual path input
    HBox([view_dropdown, index_dropdown], layout=Layout(justify_content='center')),  # Dropdowns for layout and index selection
    compute_btn,  # Button to compute indices
    HBox([prev_btn, mol_count_label, next_btn, export_indices_btn], layout=Layout(justify_content='center', gap='10px')),  # Navigation and export buttons
    HBox([output_area, indices_output], layout=Layout(justify_content='center', align_items='flex-start'))  # Output display areas
], layout=Layout(
    align_items='center', justify_content='center', width='100%'
))

# Main UI layout for the entire control panel
main_box = HBox([controls], layout=Layout(justify_content='center', align_items='center', width='100%'))
display(main_box)  # Display the UI